In [1]:
import pandas as pd
import numpy as np

In [2]:
# creating 20k random values between 1 to 20 for 2 columns
x1=np.random.randint(low=1,high=20,size=20000)


x2=np.random.randint(low=1,high=20,size=20000)

In [3]:
x1

array([ 5, 14,  2, ..., 17,  9, 14])

In [4]:
x2

array([18,  5, 12, ...,  4,  9, 12])

In [5]:
# Equation for Y (Y estimation)

#B0=3
#B1=2
#B2=-4

y=3 + 10*x1 - 40*x2 + np.random.random(20000) # adding random errors to each instance
y

array([-666.38734849,  -56.08451761, -456.40778519, ...,   13.28598947,
       -266.85281226, -336.774109  ])

you can see that we have generated data such that y is an approximate linear combination of x1 and x2, next we'll calculate optimal parameter values using gradient descent and compare them with results from sklearn and we'll see how good is the method. 

In [6]:
x=pd.DataFrame({'intercept':np.ones(x1.shape[0]),'x1':x1,'x2':x2})

In [7]:
x.shape

(20000, 3)

In [8]:
x.head()

,intercept,x1,x2
0,1.0,5,18
1,1.0,14,5
2,1.0,2,12
3,1.0,10,16
4,1.0,2,6


In [9]:
w=np.random.random(x.shape[1]) # weights or coefficients

In [10]:
w

array([0.87117247, 0.81970427, 0.05739824])

In [11]:
y

array([-666.38734849,  -56.08451761, -456.40778519, ...,   13.28598947,
       -266.85281226, -336.774109  ])

Lets write functions for predictions, error, cost and gradient that we discussed above 

In [12]:
x.head()

,intercept,x1,x2
0,1.0,5,18
1,1.0,14,5
2,1.0,2,12
3,1.0,10,16
4,1.0,2,6


In [13]:
w

array([0.87117247, 0.81970427, 0.05739824])

In [14]:
def myprediction(features,weights):

    predictions=np.dot(features,weights)
    return(predictions)

myprediction(x,w)

# function numpy.dot : is used for matrix multiplication

array([ 6.00286204, 12.6340234 ,  3.19935983, ..., 15.03573797,
        8.765095  , 13.03581104])

Note that , `np.dot` here is being used for matrix multiplication . Simple multiplication results to element wise multiplication , which is simply wrong in this context .

In [15]:
def myerror(target,features,weights):
    error=target-myprediction(features,weights)
    return(error)
myerror(y,x,w)

array([-672.39021053,  -68.71854101, -459.60714501, ...,   -1.7497485 ,
       -275.61790726, -349.80992005])

In [16]:
def mycost(target,features,weights):
    error=myerror(target,features,weights)
    cost=np.dot(error.T,error)
    return(cost)

mycost(y,x,w)

2852554090.640471

In [17]:
x.shape[0]

20000

In [18]:
def gradient(target,features,weights):
    
    error=myerror(target,features,weights)
    
    gradient=-np.dot(features.T,error)/features.shape[0]
    
    return(gradient)

gradient(y,x,w)

array([ 304.26806958, 2768.98815694, 4215.13653476])

Note that gradient here is vector of 3 values because there are 3 parameters . Also since this is being evaluated on the entire data, we scaled it down with number of observations . Do recall that , the approximation which led to the ultimate results was that change in parameters is small. We dont have any direct control over gradient , we can always chose a small value for $\eta$ to ensure that change in parameter remains small. Also if we end up chosing too small value for $\eta$, we'll need to take larger number of steps to change in parameter in order to arrive at the optimal value of the parameters 

Lets looks at the expected value for parameters from sklearn . Dont worry about the syntax here , we'll discuss that in detail, when we formally start with linear models in next module .

In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
lr=LinearRegression()
lr.fit(x.iloc[:,1:],y)
sk_estimates=([lr.intercept_]+list(lr.coef_))

In [21]:
sk_estimates

[3.5101277623905958, 9.999609579613075, -40.00052619962942]

When you run the same , these might be different for you, as we generated the data randomly 

Now lets write our version of this , using gradient descent 

In [22]:
def my_lr(target,features,learning_rate,num_steps,print_when):
    
    # start with random values of parameters
    weights=np.random.random(features.shape[1])
    
    # change parameter multiple times in sequence 
    # using the cost function gradient which we discussed earlier 
    for i in range(num_steps):
        
        weights =weights- learning_rate*gradient(target,features,weights)
       
    # this simply prints the cost function value every (print_when)th iteration
        if i%print_when==0:
            print("Output at run", i, " - ", mycost(target,features,weights),weights)
        
    return(weights)

In [23]:
my_lr(y,x,.0001,500,10)

Output at run 0  -  2696310399.056165 [ 0.03096432 -0.16976182 -0.18284896]
Output at run 10  -  1928543697.8459496 [-0.23178585 -2.46003703 -3.92592273]
Output at run 20  -  1432958617.8395758 [-0.4399563  -4.14295558 -7.02377236]
Output at run 30  -  1108984544.3704846 [-0.60488507 -5.3486192  -9.60656556]
Output at run 40  -  893461089.2868282 [ -0.73555408  -6.17997772 -11.77753502]
Output at run 50  -  746707597.3014926 [ -0.83907884  -6.71847496 -13.61857247]
Output at run 60  -  643780822.1166035 [ -0.92109622  -7.02852139 -15.19466035]
Output at run 70  -  568988479.088177 [ -0.98607182  -7.16103704 -16.55738265]
Output at run 80  -  512446051.58835006 [ -1.03754332  -7.15625821 -17.7477061 ]
Output at run 90  -  467915861.76079655 [ -1.07831333  -7.04596065 -18.79818365]
Output at run 100  -  431450841.75270486 [ -1.11060215  -6.85522077 -19.73469998]
Output at run 110  -  400543306.85067093 [ -1.13616878  -6.60381066 -20.57785445]
Output at run 120  -  373590652.44631225 [ -1

array([ -1.22388422,   4.62188081, -34.15716917])

In [24]:
sk_estimates

[3.5101277623905958, 9.999609579613075, -40.00052619962942]

you can see that if we take too few steps , we did not reach to the optimal value 

Lets increase the learning rate $\eta$

In [25]:
my_lr(y,x,.01,500,10)

Output at run 0  -  4088458322.8865066 [ -2.88160857 -27.5117754  -42.00705457]
Output at run 10  -  735760564829.148 [ -25.67130698 -273.72566791 -320.05683848]
Output at run 20  -  145722946650892.38 [ -346.90755478 -3978.53510841 -3991.04783914]
Output at run 30  -  2.8861546187014172e+16 [ -4868.5308406  -56124.20015087 -55647.0744807 ]
Output at run 40  -  5.716250376246971e+18 [ -68503.44220739 -789985.74432091 -782617.40231898]
Output at run 50  -  1.1321471882440665e+21 [  -964057.14934867 -11117836.51056523 -11013486.0291238 ]
Output at run 60  -  2.242304258006449e+23 [-1.35674504e+07 -1.56464738e+08 -1.54995529e+08]
Output at run 70  -  4.441055401349398e+25 [-1.90938736e+08 -2.20197472e+09 -2.18129746e+09]
Output at run 80  -  8.795850521815283e+27 [-2.68713735e+09 -3.09890423e+10 -3.06980444e+10]
Output at run 90  -  1.742085594757739e+30 [-3.78168795e+10 -4.36117968e+11 -4.32022668e+11]
Output at run 100  -  3.4503340091278715e+32 [-5.32208142e+11 -6.13761729e+12 -6.07998

array([3.52431663e+57, 4.06436973e+58, 4.02620388e+58])

You can see that because of high learning rate , change is parameter is huge and we end up missing the optimal point , cost function values , as well as parameter values ended up exploding. Now lets run with low learning rate and higher number of steps 

In [26]:
my_lr(y,x,.0004,100000,1000)

Output at run 0  -  2457990424.9895744 [ 0.701662   -0.70233504 -1.39203517]
Output at run 1000  -  39777.90011758702 [ -0.31327393  10.16459105 -39.83239211]
Output at run 2000  -  36014.60943594408 [ -0.11966909  10.15637288 -39.84104314]
Output at run 3000  -  32622.814027076223 [  0.06413236  10.14843488 -39.84911886]
Output at run 4000  -  29565.82138090001 [  0.2386267   10.14089884 -39.85678565]
Output at run 5000  -  26810.583251164753 [  0.40428519  10.1337444  -39.86406421]
Output at run 6000  -  24327.313570734026 [  0.56155525  10.12695223 -39.87097422]
Output at run 7000  -  22089.1664431983 [  0.71086166  10.120504   -39.87753432]
Output at run 8000  -  20071.945919799637 [  0.85260766  10.11438229 -39.88376224]
Output at run 9000  -  18253.844424166506 [  0.98717609  10.10857056 -39.8896748 ]
Output at run 10000  -  16615.206997032506 [  1.1149304   10.10305312 -39.89528797]
Output at run 11000  -  15138.318812267535 [  1.23621564  10.09781507 -39.9006169 ]
Output at run

Output at run 99000  -  1654.6945291967518 [  3.48664104  10.00062392 -39.99949426]


array([  3.48782917,  10.00057261, -39.99954646])

We can see here that we ended up getting pretty good estimates for $\beta$s , as good as from sklearn .

In [27]:
sk_estimates

[3.5101277623905958, 9.999609579613075, -40.00052619962942]

 there are modifications to gradient descent which can achieve the same thing in much less number of iterations. We'll discuss that in detail when we start with our course in Deep learning. For now ,we'll conclude this module 